In [1]:
import pandas as pd
import sqlalchemy
from time import time
from pathlib import Path
import os

In [2]:
user = 'root'
password = 'root'
host = 'localhost'
port = '5432'
database = 'ny_taxi'
table = 'green_taxi_rides'
engine = sqlalchemy.create_engine(f'postgresql://{user}:{password}'
                        f'@{host}:{port}/{database}')

In [3]:
two_levels_up = Path(os.getcwd()).parents[1].as_posix()
file_path = two_levels_up + '/data/green_tripdata_2019-09.csv.gz'

In [4]:
# Create an iterator so we can read by chunks
df_iter = pd.read_csv(file_path, iterator=True, chunksize = 100_000,
compression='gzip',)
i = 0
for df in df_iter:
    i += 1
    t_start = time()
    date_time_fields = ['lpep_pickup_datetime','lpep_dropoff_datetime', #for green taxi
                        'tpep_pickup_datetime', 'tpep_dropoff_datetime' ] # for yellow taxi
    for dt_field in date_time_fields:
        if dt_field in df.columns:
            df[dt_field] = pd.to_datetime(df[dt_field])
    df.to_sql(name = table, con = engine, if_exists = 'append')
    t_end = time()
    print(f'inserted chunK nro {i} with {df.shape[0]} rows. Took {t_end - t_start} seconds.')

inserted chunK nro 1 with 100000 rows. Took 8.881335496902466 seconds.
inserted chunK nro 2 with 100000 rows. Took 8.989350080490112 seconds.
inserted chunK nro 3 with 100000 rows. Took 8.99055290222168 seconds.


C:\Users\crist\AppData\Local\Temp\ipykernel_12004\241071527.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  for df in df_iter:


inserted chunK nro 4 with 100000 rows. Took 9.337682247161865 seconds.
inserted chunK nro 5 with 49063 rows. Took 4.194790601730347 seconds.


In [6]:
zones_file_path = file_path = two_levels_up + '/data/taxi+_zone_lookup.csv'
df_zones =  pd.read_csv(zones_file_path)
df_zones.to_sql(name = 'taxi_zones_lookup', con = engine, if_exists = 'replace')

265

In [19]:
df_zones.head(3)

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone


### Question 3. Count records

How many taxi trips were totally made on January 15?

In [7]:

query = '''
SELECT COUNT(*)
FROM green_taxi_rides
WHERE DATE(lpep_pickup_datetime) = '2019-09-18'
AND DATE(lpep_dropoff_datetime) = '2019-09-18'
'''
pd.read_sql(query, con = engine)

,count
0,15612


### Question 4. Largest trip for each day

Which was the day with the largest trip distance Use the pick up time for your calculations.

In [8]:
query = '''
SELECT DATE(lpep_pickup_datetime) AS date, trip_distance
FROM green_taxi_rides
ORDER BY trip_distance DESC
LIMIT 3
'''
pd.read_sql(query, con = engine)

,date,trip_distance
0,2019-09-26,341.64
1,2019-09-21,135.53
2,2019-09-16,114.30


### Question 5. Three biggest pick up Boroughs

Consider lpep_pickup_datetime in '2019-09-18' and ignoring Borough has Unknown

Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?

In [20]:
query = '''
SELECT pu_zone."Borough" as pickup_borough,
COUNT(*) AS q_trips, SUM(total_amount) as total_amount
FROM green_taxi_rides gtr
LEFT JOIN taxi_zones_lookup pu_zone
    ON gtr."PULocationID" = pu_zone."LocationID"
WHERE DATE(lpep_pickup_datetime) = '2019-09-18'
GROUP BY pu_zone."Borough"
HAVING SUM(total_amount) > 50000
ORDER BY total_amount DESC
LIMIT 5
'''
pd.read_sql(query, con = engine)

,pickup_borough,q_trips,total_amount
0,Brooklyn,4458,96333.24
1,Manhattan,5575,92271.30
2,Queens,4393,78671.71


### Question 6. Largest tip

For the passengers picked up in the Astoria Zone which was the drop off zone that had the largest tip? We want the name of the zone, not the id.

In [25]:
query = '''
SELECT do_zone."Zone" as dropoff_zone,
tip_amount
FROM green_taxi_rides gtr
LEFT JOIN taxi_zones_lookup pu_zone
    ON gtr."PULocationID" = pu_zone."LocationID"
LEFT JOIN taxi_zones_lookup do_zone
    ON gtr."DOLocationID" = do_zone."LocationID"
WHERE pu_zone."Zone" = 'Astoria'
ORDER BY tip_amount DESC
LIMIT 5
'''
pd.read_sql(query, con = engine)

,dropoff_zone,tip_amount
0,JFK Airport,62.31
1,Woodside,30.00
2,Kips Bay,28.00
3,NV,25.00
4,Upper West Side South,20.00
